In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from vecstack import stacking

import optuna

In [2]:
train = pd.read_csv('preprocessed_train.csv', encoding='euc-kr')
test = pd.read_csv('preprocessed_test.csv', encoding='euc-kr')

In [3]:
train

,분석데이터,label,numstrings,avlength,printables,entropy,paths,urls,registry,MZ,...,dist_86,dist_87,dist_88,dist_89,dist_90,dist_91,dist_92,dist_93,dist_94,dist_95
0,1,1,144,12.298611,1771,5.356616,0,0,0,1,...,10,4,10,9,4,0,1,0,0,0
1,2,1,804,9.580846,7703,6.063542,0,0,0,6,...,43,121,84,78,47,36,40,45,27,36
2,3,0,2205,12.736054,28083,6.107050,9,0,0,6,...,326,268,239,286,199,148,154,37,48,36
3,4,0,2602,10.288240,26770,5.373013,8,0,0,1,...,336,230,206,245,76,0,26,702,1,5
4,5,1,8980,23.252339,208806,5.775223,0,28,16,3,...,731,882,1171,1010,322,64,327,84,75,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,1,2018,13.938057,28127,5.940442,0,70,0,11,...,246,186,206,235,88,33,81,58,61,72
9996,9997,0,1105,16.437104,18163,5.766962,0,11,0,3,...,199,57,134,123,20,25,28,25,41,13
9997,9998,0,4,58.500000,234,3.811827,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9998,9999,1,3312,24.939312,82599,5.834730,0,39,0,8,...,438,985,806,851,113,123,181,100,75,86


In [4]:
x_train = train.drop(columns=['분석데이터', 'label'], axis=1)

In [5]:
y_train = train['label']

In [6]:
x_test = test.drop(columns='분석대상', axis=1)

In [7]:
"""
from xgboost import XGBClassifier

xgb_final = XGBClassifier(silent=False,
                              n_estimators = 10000,
                              booster='gbtree',
                              tree_method='gpu_hist',
                              preidctor= 'gpu_predictor',
                              scale_pos_weight=1,
                              learning_rate=0.03689407512484644,
                              objective='binary:logistic',
                              max_depth = 8,
                              subsample = 0.780714581166012,
                              colsample_bytree = 0.3723914688159835,
                              gamma = 0,
                              reg_lambda = 50.0,
                              random_state=42)
"""

"\nfrom xgboost import XGBClassifier\n\nxgb_final = XGBClassifier(silent=False,\n                              n_estimators = 10000,\n                              booster='gbtree',\n                              tree_method='gpu_hist',\n                              preidctor= 'gpu_predictor',\n                              scale_pos_weight=1,\n                              learning_rate=0.03689407512484644,\n                              objective='binary:logistic',\n                              max_depth = 8,\n                              subsample = 0.780714581166012,\n                              colsample_bytree = 0.3723914688159835,\n                              gamma = 0,\n                              reg_lambda = 50.0,\n                              random_state=42)\n"

In [8]:
# final_pred = xgb_final.fit(x_train, y_train).predict(x_test)

In [9]:
# accuracy_score(y_test, final_pred)

In [10]:
# confusion_matrix(y_test, final_pred)

In [11]:
"""
cb_model = CatBoostClassifier(objective = 'CrossEntropy',
                              colsample_bylevel = 0.07916682531951634,
                              depth = 10,
                              boosting_type = 'Plain',
                              bootstrap_type = 'Bernoulli',
                              subsample = 0.3090440812490579,
                             )
"""

"\ncb_model = CatBoostClassifier(objective = 'CrossEntropy',\n                              colsample_bylevel = 0.07916682531951634,\n                              depth = 10,\n                              boosting_type = 'Plain',\n                              bootstrap_type = 'Bernoulli',\n                              subsample = 0.3090440812490579,\n                             )\n"

In [12]:
# cb_pred = cb_model.fit(x_train, y_train).predict(x_test)

In [13]:
# accuracy_score(y_test, cb_pred)

In [14]:
# confusion_matrix(y_test, cb_pred)

In [15]:
"""
lgbm_model = LGBMClassifier(n_estimators = 10000,
                            learning_rate = 0.09416659111369403,
                            max_depth = 43,
                            boosting = 'gbdt',
                            objective = 'binary',
                            metric = 'binary_logloss',
                            is_training_metric = True,
                            num_leaves = 41,
                            min_data_in_leaf = 10,
                            feature_fraction = 0.8,
                            bagging_fraction = 0.9,
                            bagging_freq = 0,
                            alpha = 0.019782149081578264)
"""

"\nlgbm_model = LGBMClassifier(n_estimators = 10000,\n                            learning_rate = 0.09416659111369403,\n                            max_depth = 43,\n                            boosting = 'gbdt',\n                            objective = 'binary',\n                            metric = 'binary_logloss',\n                            is_training_metric = True,\n                            num_leaves = 41,\n                            min_data_in_leaf = 10,\n                            feature_fraction = 0.8,\n                            bagging_fraction = 0.9,\n                            bagging_freq = 0,\n                            alpha = 0.019782149081578264)\n"

In [16]:
# lgbm_pred = lgbm_model.fit(x_train, y_train).predict(x_test)

In [17]:
# accuracy_score(y_test, lgbm_pred)

In [18]:
# confusion_matrix(y_test, lgbm_pred)

In [19]:
xgb_params = {'n_estimators': 10000,
               'learning_rate': 0.03689407512484644,
               'max_depth': 8,
               'colsample_bytree': 0.3723914688159835,
               'subsample': 0.780714581166012,
               'eval_metric': 'auc',
               'use_label_encoder': False,
               'gamma': 0,
               'reg_lambda': 50.0,
               'tree_method': 'gpu_hist',
               'gpu_id': 0,
               'predictor': 'gpu_predictor',
               'random_state': 42 }

lgb_params = {'n_estimators': 10000,
              'learning_rate':0.09416659111369403,
              'max_depth':43,
              'boosting':'gbdt',
              'objective': 'binary',
              'metric': 'binary_logloss',
              'is_training_metric': True,
              'num_leaves':41,
              'min_data_in_leaf':10,
              'feature_fraction':0.8,
              'bagging_fraction':0.9,
              'bagging_freq':0,
              'alpha': 0.019782149081578264 }

cat_params = {'objective': 'CrossEntropy',
              'colsample_bylevel': 0.07916682531951634,
              'depth': 10,
              'boosting_type': 'Plain',
              'bootstrap_type': 'Bernoulli',
              'subsample': 0.3090440812490579 }

In [20]:
lgbm = LGBMClassifier(**lgb_params)

xgb = XGBClassifier(**xgb_params)

cat = CatBoostClassifier(**cat_params)

In [21]:
models = [lgbm, xgb, cat]

In [22]:
S_train, S_test = stacking(models, x_train, y_train, x_test, regression=False,
                          metric=accuracy_score, n_folds=5, stratified=True, shuffle=True, random_state=42, verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LGBMClassifier]
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
    fold  0:  [0.93200000]
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[Li

92:	learn: 0.3060257	total: 11.4s	remaining: 1m 50s
93:	learn: 0.3048429	total: 11.5s	remaining: 1m 50s
94:	learn: 0.3038298	total: 11.6s	remaining: 1m 50s
95:	learn: 0.3026470	total: 11.8s	remaining: 1m 50s
96:	learn: 0.3014429	total: 11.9s	remaining: 1m 50s
97:	learn: 0.2996875	total: 12s	remaining: 1m 50s
98:	learn: 0.2987485	total: 12.2s	remaining: 1m 50s
99:	learn: 0.2976294	total: 12.3s	remaining: 1m 50s
100:	learn: 0.2968061	total: 12.4s	remaining: 1m 50s
101:	learn: 0.2957415	total: 12.6s	remaining: 1m 50s
102:	learn: 0.2944935	total: 12.7s	remaining: 1m 50s
103:	learn: 0.2932678	total: 12.8s	remaining: 1m 50s
104:	learn: 0.2921107	total: 13s	remaining: 1m 50s
105:	learn: 0.2913056	total: 13.1s	remaining: 1m 50s
106:	learn: 0.2900754	total: 13.2s	remaining: 1m 50s
107:	learn: 0.2894090	total: 13.3s	remaining: 1m 50s
108:	learn: 0.2886174	total: 13.4s	remaining: 1m 49s
109:	learn: 0.2877877	total: 13.5s	remaining: 1m 49s
110:	learn: 0.2873162	total: 13.7s	remaining: 1m 49s
111:	

248:	learn: 0.1953187	total: 30.7s	remaining: 1m 32s
249:	learn: 0.1949811	total: 30.8s	remaining: 1m 32s
250:	learn: 0.1944480	total: 31s	remaining: 1m 32s
251:	learn: 0.1939709	total: 31.1s	remaining: 1m 32s
252:	learn: 0.1936220	total: 31.2s	remaining: 1m 32s
253:	learn: 0.1932608	total: 31.4s	remaining: 1m 32s
254:	learn: 0.1929074	total: 31.5s	remaining: 1m 31s
255:	learn: 0.1924130	total: 31.6s	remaining: 1m 31s
256:	learn: 0.1921296	total: 31.7s	remaining: 1m 31s
257:	learn: 0.1916632	total: 31.8s	remaining: 1m 31s
258:	learn: 0.1908763	total: 32s	remaining: 1m 31s
259:	learn: 0.1905592	total: 32.1s	remaining: 1m 31s
260:	learn: 0.1898869	total: 32.2s	remaining: 1m 31s
261:	learn: 0.1892392	total: 32.4s	remaining: 1m 31s
262:	learn: 0.1889541	total: 32.5s	remaining: 1m 30s
263:	learn: 0.1883790	total: 32.6s	remaining: 1m 30s
264:	learn: 0.1878893	total: 32.7s	remaining: 1m 30s
265:	learn: 0.1873034	total: 32.8s	remaining: 1m 30s
266:	learn: 0.1866551	total: 32.9s	remaining: 1m 3

404:	learn: 0.1297660	total: 49.7s	remaining: 1m 13s
405:	learn: 0.1295509	total: 49.8s	remaining: 1m 12s
406:	learn: 0.1293743	total: 50s	remaining: 1m 12s
407:	learn: 0.1292336	total: 50.1s	remaining: 1m 12s
408:	learn: 0.1290253	total: 50.2s	remaining: 1m 12s
409:	learn: 0.1287655	total: 50.3s	remaining: 1m 12s
410:	learn: 0.1282635	total: 50.5s	remaining: 1m 12s
411:	learn: 0.1278089	total: 50.6s	remaining: 1m 12s
412:	learn: 0.1276177	total: 50.7s	remaining: 1m 12s
413:	learn: 0.1274346	total: 50.8s	remaining: 1m 11s
414:	learn: 0.1269948	total: 50.9s	remaining: 1m 11s
415:	learn: 0.1265937	total: 51s	remaining: 1m 11s
416:	learn: 0.1264464	total: 51.2s	remaining: 1m 11s
417:	learn: 0.1262054	total: 51.3s	remaining: 1m 11s
418:	learn: 0.1259626	total: 51.4s	remaining: 1m 11s
419:	learn: 0.1256073	total: 51.6s	remaining: 1m 11s
420:	learn: 0.1253152	total: 51.7s	remaining: 1m 11s
421:	learn: 0.1251925	total: 51.8s	remaining: 1m 10s
422:	learn: 0.1248564	total: 51.9s	remaining: 1m 1

564:	learn: 0.0862716	total: 1m 9s	remaining: 53.6s
565:	learn: 0.0860809	total: 1m 9s	remaining: 53.5s
566:	learn: 0.0858912	total: 1m 9s	remaining: 53.4s
567:	learn: 0.0857622	total: 1m 10s	remaining: 53.3s
568:	learn: 0.0855462	total: 1m 10s	remaining: 53.1s
569:	learn: 0.0854757	total: 1m 10s	remaining: 53s
570:	learn: 0.0853800	total: 1m 10s	remaining: 52.9s
571:	learn: 0.0852213	total: 1m 10s	remaining: 52.8s
572:	learn: 0.0849796	total: 1m 10s	remaining: 52.6s
573:	learn: 0.0849126	total: 1m 10s	remaining: 52.5s
574:	learn: 0.0846700	total: 1m 10s	remaining: 52.4s
575:	learn: 0.0845081	total: 1m 10s	remaining: 52.3s
576:	learn: 0.0844373	total: 1m 11s	remaining: 52.1s
577:	learn: 0.0841967	total: 1m 11s	remaining: 52s
578:	learn: 0.0839064	total: 1m 11s	remaining: 51.9s
579:	learn: 0.0836537	total: 1m 11s	remaining: 51.7s
580:	learn: 0.0834801	total: 1m 11s	remaining: 51.6s
581:	learn: 0.0832643	total: 1m 11s	remaining: 51.5s
582:	learn: 0.0829430	total: 1m 11s	remaining: 51.4s


...

333:	learn: 0.1578839	total: 40.5s	remaining: 1m 20s
334:	learn: 0.1574406	total: 40.7s	remaining: 1m 20s
335:	learn: 0.1569875	total: 40.8s	remaining: 1m 20s
336:	learn: 0.1565964	total: 40.9s	remaining: 1m 20s
337:	learn: 0.1560630	total: 41s	remaining: 1m 20s
338:	learn: 0.1553715	total: 41.1s	remaining: 1m 20s
339:	learn: 0.1548172	total: 41.3s	remaining: 1m 20s
340:	learn: 0.1545105	total: 41.4s	remaining: 1m 19s
341:	learn: 0.1543920	total: 41.5s	remaining: 1m 19s
342:	learn: 0.1536620	total: 41.7s	remaining: 1m 19s
343:	learn: 0.1533200	total: 41.8s	remaining: 1m 19s
344:	learn: 0.1529659	total: 41.9s	remaining: 1m 19s
345:	learn: 0.1525652	total: 42s	remaining: 1m 19s
346:	learn: 0.1521045	total: 42.1s	remaining: 1m 19s
347:	learn: 0.1517141	total: 42.2s	remaining: 1m 19s
348:	learn: 0.1513256	total: 42.4s	remaining: 1m 19s
349:	learn: 0.1510173	total: 42.5s	remaining: 1m 18s
350:	learn: 0.1508697	total: 42.6s	remaining: 1m 18s
351:	learn: 0.1504520	total: 42.7s	remaining: 1m 1

490:	learn: 0.1037508	total: 59.6s	remaining: 1m 1s
491:	learn: 0.1036335	total: 59.7s	remaining: 1m 1s
492:	learn: 0.1032816	total: 59.9s	remaining: 1m 1s
493:	learn: 0.1030814	total: 60s	remaining: 1m 1s
494:	learn: 0.1028985	total: 1m	remaining: 1m 1s
495:	learn: 0.1023409	total: 1m	remaining: 1m 1s
496:	learn: 0.1021752	total: 1m	remaining: 1m 1s
497:	learn: 0.1019204	total: 1m	remaining: 1m
498:	learn: 0.1017092	total: 1m	remaining: 1m
499:	learn: 0.1013317	total: 1m	remaining: 1m
500:	learn: 0.1011847	total: 1m	remaining: 1m
501:	learn: 0.1009850	total: 1m	remaining: 1m
502:	learn: 0.1007026	total: 1m 1s	remaining: 1m
503:	learn: 0.1004131	total: 1m 1s	remaining: 1m
504:	learn: 0.1002335	total: 1m 1s	remaining: 1m
505:	learn: 0.1000004	total: 1m 1s	remaining: 60s
506:	learn: 0.0997703	total: 1m 1s	remaining: 59.9s
507:	learn: 0.0995481	total: 1m 1s	remaining: 59.8s
508:	learn: 0.0993137	total: 1m 1s	remaining: 59.7s
509:	learn: 0.0988447	total: 1m 1s	remaining: 59.5s
510:	learn: 

648:	learn: 0.0728948	total: 1m 18s	remaining: 42.5s
649:	learn: 0.0727599	total: 1m 18s	remaining: 42.4s
650:	learn: 0.0726366	total: 1m 18s	remaining: 42.3s
651:	learn: 0.0725029	total: 1m 19s	remaining: 42.2s
652:	learn: 0.0724543	total: 1m 19s	remaining: 42.1s
653:	learn: 0.0723226	total: 1m 19s	remaining: 41.9s
654:	learn: 0.0721712	total: 1m 19s	remaining: 41.8s
655:	learn: 0.0720266	total: 1m 19s	remaining: 41.7s
656:	learn: 0.0719234	total: 1m 19s	remaining: 41.6s
657:	learn: 0.0718131	total: 1m 19s	remaining: 41.4s
658:	learn: 0.0716386	total: 1m 19s	remaining: 41.3s
659:	learn: 0.0715206	total: 1m 19s	remaining: 41.2s
660:	learn: 0.0714260	total: 1m 20s	remaining: 41.1s
661:	learn: 0.0711131	total: 1m 20s	remaining: 40.9s
662:	learn: 0.0709318	total: 1m 20s	remaining: 40.8s
663:	learn: 0.0707824	total: 1m 20s	remaining: 40.7s
664:	learn: 0.0705726	total: 1m 20s	remaining: 40.6s
665:	learn: 0.0703931	total: 1m 20s	remaining: 40.4s
666:	learn: 0.0703141	total: 1m 20s	remaining:

804:	learn: 0.0536260	total: 1m 37s	remaining: 23.6s
805:	learn: 0.0535637	total: 1m 37s	remaining: 23.5s
806:	learn: 0.0534266	total: 1m 37s	remaining: 23.4s
807:	learn: 0.0533161	total: 1m 37s	remaining: 23.3s
808:	learn: 0.0531639	total: 1m 38s	remaining: 23.2s
809:	learn: 0.0530107	total: 1m 38s	remaining: 23s
810:	learn: 0.0529357	total: 1m 38s	remaining: 22.9s
811:	learn: 0.0528792	total: 1m 38s	remaining: 22.8s
812:	learn: 0.0528046	total: 1m 38s	remaining: 22.7s
813:	learn: 0.0527034	total: 1m 38s	remaining: 22.6s
814:	learn: 0.0526456	total: 1m 38s	remaining: 22.4s
815:	learn: 0.0525652	total: 1m 38s	remaining: 22.3s
816:	learn: 0.0525303	total: 1m 39s	remaining: 22.2s
817:	learn: 0.0524408	total: 1m 39s	remaining: 22.1s
818:	learn: 0.0523072	total: 1m 39s	remaining: 22s
819:	learn: 0.0522003	total: 1m 39s	remaining: 21.8s
820:	learn: 0.0521296	total: 1m 39s	remaining: 21.7s
821:	learn: 0.0520801	total: 1m 39s	remaining: 21.6s
822:	learn: 0.0519573	total: 1m 39s	remaining: 21.

960:	learn: 0.0402538	total: 1m 56s	remaining: 4.74s
961:	learn: 0.0401933	total: 1m 56s	remaining: 4.61s
962:	learn: 0.0401473	total: 1m 56s	remaining: 4.49s
963:	learn: 0.0400993	total: 1m 57s	remaining: 4.37s
964:	learn: 0.0400069	total: 1m 57s	remaining: 4.25s
965:	learn: 0.0399261	total: 1m 57s	remaining: 4.13s
966:	learn: 0.0398704	total: 1m 57s	remaining: 4s
967:	learn: 0.0398262	total: 1m 57s	remaining: 3.88s
968:	learn: 0.0397486	total: 1m 57s	remaining: 3.76s
969:	learn: 0.0396141	total: 1m 57s	remaining: 3.64s
970:	learn: 0.0395815	total: 1m 57s	remaining: 3.52s
971:	learn: 0.0395236	total: 1m 57s	remaining: 3.4s
972:	learn: 0.0394758	total: 1m 58s	remaining: 3.28s
973:	learn: 0.0394079	total: 1m 58s	remaining: 3.15s
974:	learn: 0.0393655	total: 1m 58s	remaining: 3.03s
975:	learn: 0.0393295	total: 1m 58s	remaining: 2.91s
976:	learn: 0.0392768	total: 1m 58s	remaining: 2.79s
977:	learn: 0.0392183	total: 1m 58s	remaining: 2.67s
978:	learn: 0.0391753	total: 1m 58s	remaining: 2.5

In [23]:
model = lgbm

In [24]:
model = model.fit(S_train, y_train)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0


In [25]:
y_pred = model.predict(S_test)

In [27]:
# print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

In [28]:
# confusion_matrix(y_test, y_pred)

In [29]:
y_pred

array([1, 1, 1, ..., 0, 1, 1], dtype=int64)

In [31]:
y_pred.shape

(5000,)

In [30]:
sub = pd.read_csv('submission_ex.csv', encoding='euc-kr')
sub['정답지'] = y_pred
sub.to_csv('sub.csv', index = 0, encoding='euc-kr')